In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sjögrens_Syndrome"
cohort = "GSE143153"

# Input paths
in_trait_dir = "../../input/GEO/Sjögrens_Syndrome"
in_cohort_dir = "../../input/GEO/Sjögrens_Syndrome/GSE143153"

# Output paths
out_data_file = "../../output/preprocess/Sjögrens_Syndrome/GSE143153.csv"
out_gene_data_file = "../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE143153.csv"
out_clinical_data_file = "../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE143153.csv"
json_path = "../../output/preprocess/Sjögrens_Syndrome/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Microarray analysis of salivary gland CD4+ T cells"
!Series_summary	"Whole human genome arrays were used to assess the transcriptome differences in CD3+CD4+CD45RA- memory T cells isolated and sorted from minor salivary gland biopsy tissue of individuals who met 2002 American European Consensus Group classification criteria for primary Sjogren’s syndrome (SS) and subjects who did not meet criteria for SS, lacked focal lymphocytic sialoadenitis, lacked anti-Ro antibodies, lacked anti-La antibodies, but who had subjective symptoms of dryness (non-SS, sicca controls)."
!Series_overall_design	"Samples from 17 pSS and 15 non-SS subjects were hybridized to Agilent Whole Human Genome 8x60K microarrays in three batches (Batch 1: 2 pSS, 3 non-SS; Batch 2: 6 pSS, 5 non-SS; Batch 3: 9 pSS, 7 non-SS).  All data were pooled to assess potential batch effects by principal components analysis and gene expression data were quality control checked using the arrayQua

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import json
import pandas as pd
from typing import Optional, Callable, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait: From the sample characteristics dictionary, key 1 contains disease classification
trait_row = 1

# For age: From the sample characteristics dictionary, key 2 contains age information
age_row = 2

# For gender: From the sample characteristics dictionary, key 3 contains sex information
gender_row = 3

# 2.2 Data Type Conversion
# Function for trait conversion
def convert_trait(value):
    # Extract the value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: Primary SS = 1, non-SS = 0
    if "Primary SS" in value:
        return 1
    elif "non-SS" in value:
        return 0
    else:
        return None

# Function for age conversion
def convert_age(value):
    # Extract the value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to numeric (continuous)
    try:
        return float(value)
    except:
        return None

# Function for gender conversion
def convert_gender(value):
    # Extract the value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: M = 1, F = 0
    if value.upper() == "M":
        return 1
    elif value.upper() == "F":
        return 0
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Save cohort information for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (Only if trait_row is not None)
# At this analysis step, we're examining the availability of data rather than
# processing actual files, which will be done in later steps
if trait_row is not None:
    print(f"Trait data (Sjögrens_Syndrome) available at row {trait_row}")
    if age_row is not None:
        print(f"Age data available at row {age_row}")
    if gender_row is not None:
        print(f"Gender data available at row {gender_row}")
    
    print("Clinical data conversion functions defined:")
    print("- convert_trait: Converts 'Primary SS' to 1, 'non-SS' to 0")
    print("- convert_age: Converts age values to float")
    print("- convert_gender: Converts 'M' to 1, 'F' to 0")


Trait data (Sjögrens_Syndrome) available at row 1
Age data available at row 2
Gender data available at row 3
Clinical data conversion functions defined:
- convert_trait: Converts 'Primary SS' to 1, 'non-SS' to 0
- convert_age: Converts age values to float
- convert_gender: Converts 'M' to 1, 'F' to 0


### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these appear to be simple numeric identifiers (1, 2, 3, etc.)
# rather than human gene symbols (like BRCA1, TP53, etc.).
# In GEO datasets, numeric identifiers typically need to be mapped to gene symbols
# using the platform annotation information.

# These numeric identifiers are not standard human gene symbols and will need mapping
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'ProbeName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'A_23_P326296', 'A_24_P287941'], 'GB_ACC': [nan, nan, nan, 'NM_144987', 'NM_013290'], 'ControlType': [1.0, 1.0, 1.0, 0.0, 0.0], 'accessions': [nan, nan, nan, 'ref|NM_144987|ref|NM_001040425|ens|ENST00000292879|ens|ENST00000392196', 'ref|NM_013290|ref|NM_016556|ens|ENST00000393795|ens|ENST00000253789'], 'GeneName': ['GE_BrightCorner', 'DarkCorner', 'DarkCorner', 'U2AF1L4', 'PSMC3IP'], 'Description': [nan, nan, nan, 'ref|Homo sapiens U2 small nuclear RNA auxiliary factor 1-like 4 (U2AF1L4), transcript variant 2, mRNA [NM_144987]', 'ref|Homo sapiens PSMC3 interacting protein (PSMC3IP), transcript variant 1, mRNA [NM_013290]'], 'chr_coord': [nan, nan, nan, 'hs|chr19:036235296-036235237', 'hs|chr17:040724775-040724716'], 'SEQUENCE': [nan, nan, nan, 'GTATGGGGAGATTGAAGAGATGAATGTGTGCGACAACCTTGGGGACCACGTCGTGGGCAA', 'AAATTGCAGTAGCTTGAGGTTAACATTTAGACTTGGAACAATGCTAAAGGAAAGCATTTG'

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns for mapping
# From the preview, we can see that:
# - 'ID' column contains numeric identifiers matching the gene expression data index
# - 'GeneName' column contains the human gene symbols we want to map to

# 2. Get a gene mapping dataframe
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'GeneName'  # Column containing gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Let's examine the first few rows of the gene expression data after mapping
print("Gene expression data after mapping (first 5 rows):")
print(gene_data.head())

# Check the shape of the resulting gene expression data
print(f"\nShape of gene expression data: {gene_data.shape}")


Gene expression data after mapping (first 5 rows):
       GSM4251021  GSM4251022  GSM4251023  GSM4251024  GSM4251025  GSM4251026  \
Gene                                                                            
A1BG     4.960248    8.602593    3.865823    4.940824    3.691263    6.169120   
A1CF    43.791353   95.789151   95.931706   94.097990   87.790806   55.940933   
A2BP1   10.506252   13.071567   15.081290   16.994973   13.472849   10.773222   
A2LD1    2.345080    2.535318    2.443681    2.792124    2.568849    2.539543   
A2M     65.219899   95.236352   28.492692   37.175936   98.877802   96.035878   

       GSM4251027  GSM4251028  GSM4251029  GSM4251030  ...  GSM4251043  \
Gene                                                   ...               
A1BG     3.047159    4.265922    9.047001    2.880536  ...    5.092730   
A1CF    53.160565   52.885941   45.794629   71.312747  ...   51.482496   
A2BP1    9.149361    9.223923    9.372810   11.239728  ...    8.634298   
A2LD1    2.

### Step 7: Data Normalization and Linking

In [8]:
# 1. Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_data, 
    trait=trait, 
    trait_row=trait_row, 
    convert_trait=convert_trait,
    age_row=age_row, 
    convert_age=convert_age,
    gender_row=gender_row, 
    convert_gender=convert_gender
)

# Save the clinical features data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Normalized gene data shape: {normalized_gene_data.shape}")
print(f"First few normalized gene symbols: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print(linked_data.head())

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note=f"Dataset contains gene expression data from CD4 T-cells of pSS patients and healthy controls."
)

# 6. Save the data if it's usable
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    # Save the data
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data quality check failed. The dataset is not suitable for association studies.")

Clinical data saved to ../../output/preprocess/Sjögrens_Syndrome/clinical_data/GSE143153.csv
Normalized gene data shape: (19274, 32)
First few normalized gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS']


Normalized gene data saved to ../../output/preprocess/Sjögrens_Syndrome/gene_data/GSE143153.csv
Linked data shape: (32, 19277)
            Sjögrens_Syndrome   Age  Gender      A1BG  A1BG-AS1       A1CF  \
GSM4251021                1.0  56.0     1.0  4.960248  4.592398  43.791353   
GSM4251022                1.0  51.0     0.0  8.602593  3.327157  95.789151   
GSM4251023                0.0  37.0     0.0  3.865823  5.551529  95.931706   
GSM4251024                0.0  40.0     0.0  4.940824  3.408210  94.097990   
GSM4251025                0.0  41.0     0.0  3.691263  5.024266  87.790806   

                  A2M     A2ML1     A4GALT     A4GNT  ...    ZWILCH  \
GSM4251021  65.219899  3.988163  15.353936  2.987036  ...  5.716409   
GSM4251022  95.236352  2.219424  16.845896  4.322902  ...  3.196549   
GSM4251023  28.492692  4.329435  18.440743  7.215498  ...  4.035570   
GSM4251024  37.175936  4.833797  17.263753  3.104170  ...  2.323874   
GSM4251025  98.877802  4.673251  13.087190  4.312

Shape after handling missing values: (32, 19277)
For the feature 'Sjögrens_Syndrome', the least common label is '0.0' with 15 occurrences. This represents 46.88% of the dataset.
The distribution of the feature 'Sjögrens_Syndrome' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.0
  50% (Median): 45.5
  75%: 58.0
Min: 19.0
Max: 71.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 3 occurrences. This represents 9.38% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.



Linked data saved to ../../output/preprocess/Sjögrens_Syndrome/GSE143153.csv
